## Predict on Harvey

In [ ]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
import transformers
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import KFold
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score
import pandas as pd
import numpy as np
from tqdm import tqdm
import copy
import math

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.manual_seed(64)

# Load the saved models
save_path = 'Training_results/Harvey_5foldcv_BERT_Multihead/'
max_length = 256
batchsize = 48
saved_model_paths = [os.path.join(save_path, f"best_bert_model_fold_{i+1}.pth") for i in range(5)]
models = []

# Load BERT tokenizer and model
tokenizer = transformers.BertTokenizer.from_pretrained('bert-base-uncased')
model_bert = transformers.BertModel.from_pretrained('bert-base-uncased')

# Get the target list
target_list = [
    'homeOwnersInsurance', 'floodInsurance', 'destroyed', 'floodDamage', 'roofDamage',
    'tsaEligible', 'tsaCheckedIn', 'rentalAssistanceEligible', 'repairAssistanceEligible',
    'replacementAssistanceEligible', 'personalPropertyEligible'
]

def haversine(lat1, lon1, lat2, lon2):
    R = 3959
    lat1, lon1, lat2, lon2 = map(math.radians, [lat1, lon1, lat2, lon2])
    dlat, dlon = lat2 - lat1, lon2 - lon1
    a = math.sin(dlat / 2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon / 2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    return R * c

def get_box_area(lat1, lon1, lat2, lon2):
    side1 = haversine(lat1, lon1, lat1, lon2)
    side2 = haversine(lat1, lon1, lat2, lon1)
    return side1 * side2

# Get the target list if not provided
def get_target_list(target_list=[]):
    if not target_list:
        target_list = [
            'homeOwnersInsurance', 'floodInsurance', 'destroyed', 'floodDamage', 'roofDamage',
            'tsaEligible', 'tsaCheckedIn', 'rentalAssistanceEligible', 'repairAssistanceEligible',
            'replacementAssistanceEligible', 'personalPropertyEligible'
        ]
    return target_list

def grouped_tweets_to_dict(tweet_grouped):
    return {int(name): group['text'] for name, group in tweet_grouped}

# CustomDataset without labels (for inference only)
class InferenceDataset(Dataset):
    def __init__(self, texts, tokenizer, max_len=512):
        self.texts = texts
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_len,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )
        return {
            'input_ids': encoding['input_ids'].squeeze(0),
            'attention_mask': encoding['attention_mask'].squeeze(0)
        }

class BERTDeepMultiHeadClassifier(nn.Module):
    def __init__(self, num_targets=11, hidden_dim=256):
        super(BERTDeepMultiHeadClassifier, self).__init__()
        self.bert = model_bert
        self.drop = nn.Dropout(0.3)

        self.heads = nn.ModuleList([
            nn.Sequential(
                nn.Linear(self.bert.config.hidden_size, hidden_dim),
                nn.ReLU(),
                nn.Linear(hidden_dim, 1)
            ) for _ in range(num_targets)
        ])

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs.pooler_output
        x = self.drop(pooled_output)
        return torch.cat([torch.sigmoid(head(x)) for head in self.heads], dim=1)

# Load all trained models
for path in saved_model_paths:
    model = BERTDeepMultiHeadClassifier(num_targets=len(target_list)).to(device)
    model.load_state_dict(torch.load(path))
    model.eval()
    models.append(model)

size_threshold = 80

tweets_harvey2 = pd.read_csv('D:/TAMIDS/Disaster_Impact_Estimation/tweets/harvey_corrected.csv')
tweets_harvey2.rename(columns={'zipcode': 'zip_code'}, inplace=True)

bboxes_useful = tweets_harvey2.place_bbox.apply(lambda x: [[float(i.strip('()[]')) for i in x.split(', ')][i] for i in [1,0,3,2]])
bbu_areas = bboxes_useful.apply(lambda x: get_box_area(*x))
tweets_harvey = tweets_harvey2.loc[((tweets_harvey2.geo.apply(lambda x: 'Point' in str(x))) | (bbu_areas < size_threshold)), :]
tweets_harvey.loc[:, 'zip_code'] = tweets_harvey['zip_code'].apply(int)
tweet_grouped_harvey = tweets_harvey.groupby('zip_code')

tweet_dict = {int(name): '\n'.join(group['text'].to_list()) for name, group in tweet_grouped_harvey}

# Prepare data
zip_codes = list(tweet_dict.keys())
texts = [tweet_dict[zc] for zc in zip_codes]

# Create inference dataset and loader
inference_dataset = InferenceDataset(texts=texts, tokenizer=tokenizer, max_len=max_length)
inference_loader = DataLoader(inference_dataset, batch_size=batchsize, shuffle=False)

# Store predictions
all_preds = []

with torch.no_grad():
    for batch in tqdm(inference_loader, desc="Making predictions", unit="batch"):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)

        fold_preds = []
        for model in models:
            outputs = model(input_ids, attention_mask)
            fold_preds.append(outputs.cpu().numpy())

        fold_preds = np.stack(fold_preds, axis=0)  # Shape: (num_models, batch_size, num_targets)
        all_preds.append(fold_preds)

# Combine predictions
all_preds = np.concatenate(all_preds, axis=1)  # Shape: (num_models, total_samples, num_targets)
mean_preds = all_preds.mean(axis=0)            # Shape: (total_samples, num_targets)
binary_preds = (mean_preds > 0.5).astype(int)

target_list = [
    'homeOwnersInsurance', 'floodInsurance', 'destroyed', 'floodDamage', 'roofDamage',
    'tsaEligible', 'tsaCheckedIn', 'rentalAssistanceEligible', 'repairAssistanceEligible',
    'replacementAssistanceEligible', 'personalPropertyEligible'
]

# Save to files
mean_preds_df = pd.DataFrame(mean_preds, columns=target_list)
mean_preds_df.insert(0, "zip_code", zip_codes)
mean_preds_path = os.path.join(save_path, "all_zipcodes_with_tweets_bert_mean_predictions.xlsx")
mean_preds_df.to_excel(mean_preds_path, index=False)

binary_preds_df = pd.DataFrame(binary_preds, columns=target_list)
binary_preds_df.insert(0, "zip_code", zip_codes)
binary_preds_path = os.path.join(save_path, "all_zipcodes_with_tweets_bert_binary_predictions.xlsx")
binary_preds_df.to_excel(binary_preds_path, index=False)

print(f"Mean predictions saved to {mean_preds_path}")
print(f"Binary predictions saved to {binary_preds_path}")

C:\Users\udays\AppData\Local\Temp\ipykernel_18096\1686881798.py:110: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(path))
Making predictions

Mean predictions saved to Training_results/Harvey_5foldcv_BERT_Multihead/all_zipcodes_with_tweets_bert_mean_predictions.xlsx
Binary predictions saved to Training_results/Harvey_5foldcv_BERT_Multihead/all_zipcodes_with_tweets_bert_binary_predictions.xlsx


In [ ]:
for model in models:
    del model

## Predict on Beryl

In [ ]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
import transformers
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import KFold
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score
import pandas as pd
import numpy as np
from tqdm import tqdm
import copy
import math

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.manual_seed(64)

# Load the saved models
save_path = 'Training_results/Beryl_5foldcv_BERT_Multihead_256/'
max_length = 256
batchsize = 48
saved_model_paths = [os.path.join(save_path, f"best_bert_model_fold_{i+1}.pth") for i in range(5)]
models = []

# Load BERT tokenizer and model
tokenizer = transformers.BertTokenizer.from_pretrained('bert-base-uncased')
model_bert = transformers.BertModel.from_pretrained('bert-base-uncased')

# Get the target list
target_list = [
    'homeOwnersInsurance', 'floodInsurance', 'destroyed', 'floodDamage', 'roofDamage',
    'tsaEligible', 'tsaCheckedIn', 'rentalAssistanceEligible', 'repairAssistanceEligible',
    'replacementAssistanceEligible', 'personalPropertyEligible'
]

def haversine(lat1, lon1, lat2, lon2):
    R = 3959
    lat1, lon1, lat2, lon2 = map(math.radians, [lat1, lon1, lat2, lon2])
    dlat, dlon = lat2 - lat1, lon2 - lon1
    a = math.sin(dlat / 2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon / 2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    return R * c

def get_box_area(lat1, lon1, lat2, lon2):
    side1 = haversine(lat1, lon1, lat1, lon2)
    side2 = haversine(lat1, lon1, lat2, lon1)
    return side1 * side2

# Get the target list if not provided
def get_target_list(target_list=[]):
    if not target_list:
        target_list = [
            'homeOwnersInsurance', 'floodInsurance', 'destroyed', 'floodDamage', 'roofDamage',
            'tsaEligible', 'tsaCheckedIn', 'rentalAssistanceEligible', 'repairAssistanceEligible',
            'replacementAssistanceEligible', 'personalPropertyEligible'
        ]
    return target_list

def grouped_tweets_to_dict(tweet_grouped):
    return {int(name): group['text'] for name, group in tweet_grouped}

# CustomDataset without labels (for inference only)
class InferenceDataset(Dataset):
    def __init__(self, texts, tokenizer, max_len=512):
        self.texts = texts
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_len,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )
        return {
            'input_ids': encoding['input_ids'].squeeze(0),
            'attention_mask': encoding['attention_mask'].squeeze(0)
        }

class BERTDeepMultiHeadClassifier(nn.Module):
    def __init__(self, num_targets=11, hidden_dim=256):
        super(BERTDeepMultiHeadClassifier, self).__init__()
        self.bert = model_bert
        self.drop = nn.Dropout(0.3)

        self.heads = nn.ModuleList([
            nn.Sequential(
                nn.Linear(self.bert.config.hidden_size, hidden_dim),
                nn.ReLU(),
                nn.Linear(hidden_dim, 1)
            ) for _ in range(num_targets)
        ])

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs.pooler_output
        x = self.drop(pooled_output)
        return torch.cat([torch.sigmoid(head(x)) for head in self.heads], dim=1)

# Load all trained models
for path in saved_model_paths:
    model = BERTDeepMultiHeadClassifier(num_targets=len(target_list)).to(device)
    model.load_state_dict(torch.load(path))
    model.eval()
    models.append(model)

size_threshold = 80

tweets2 = pd.read_csv('D:/TAMIDS/Disaster_Impact_Estimation/tweets/organized_with_zipcode.csv')  # read in massive tweets dataset
tweets_beryl = tweets2[(tweets2.storm_name == 'beryl')]
bboxes_useful = tweets_beryl.place_bbox.apply(lambda x: [[float(i.strip('(').strip(')')) for i in x.split(', ')][i] for i in [1,0,3,2]])
bbu_areas = bboxes_useful.apply(lambda x: get_box_area(*x))
tweets_beryl = tweets_beryl.loc[((tweets_beryl.geo.apply(lambda x: 'Point' in str(x))) | (bbu_areas < size_threshold)),:]  # since i'm using iloc i think the indices will match
tweet_grouped_beryl = tweets_beryl.groupby('zip_code')

tweet_dict = {int(name): '\n'.join(group['text'].to_list()) for name, group in tweet_grouped_beryl}

# Prepare data
zip_codes = list(tweet_dict.keys())
texts = [tweet_dict[zc] for zc in zip_codes]

# Create inference dataset and loader
inference_dataset = InferenceDataset(texts=texts, tokenizer=tokenizer, max_len=max_length)
inference_loader = DataLoader(inference_dataset, batch_size=batchsize, shuffle=False)

# Store predictions
all_preds = []

with torch.no_grad():
    for batch in tqdm(inference_loader, desc="Making predictions", unit="batch"):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)

        fold_preds = []
        for model in models:
            outputs = model(input_ids, attention_mask)
            fold_preds.append(outputs.cpu().numpy())

        fold_preds = np.stack(fold_preds, axis=0)  # Shape: (num_models, batch_size, num_targets)
        all_preds.append(fold_preds)

# Combine predictions
all_preds = np.concatenate(all_preds, axis=1)  # Shape: (num_models, total_samples, num_targets)
mean_preds = all_preds.mean(axis=0)            # Shape: (total_samples, num_targets)
binary_preds = (mean_preds > 0.5).astype(int)

target_list = [
    'homeOwnersInsurance', 'floodInsurance', 'destroyed', 'floodDamage', 'roofDamage',
    'tsaEligible', 'tsaCheckedIn', 'rentalAssistanceEligible', 'repairAssistanceEligible',
    'replacementAssistanceEligible', 'personalPropertyEligible'
]

# Save to files
mean_preds_df = pd.DataFrame(mean_preds, columns=target_list)
mean_preds_df.insert(0, "zip_code", zip_codes)
mean_preds_path = os.path.join(save_path, "all_zipcodes_with_tweets_bert_mean_predictions.xlsx")
mean_preds_df.to_excel(mean_preds_path, index=False)

binary_preds_df = pd.DataFrame(binary_preds, columns=target_list)
binary_preds_df.insert(0, "zip_code", zip_codes)
binary_preds_path = os.path.join(save_path, "all_zipcodes_with_tweets_bert_binary_predictions.xlsx")
binary_preds_df.to_excel(binary_preds_path, index=False)

print(f"Mean predictions saved to {mean_preds_path}")
print(f"Binary predictions saved to {binary_preds_path}")

C:\Users\udays\AppData\Local\Temp\ipykernel_18096\2878012014.py:110: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(path))
Making predictions

Mean predictions saved to Training_results/Beryl_5foldcv_BERT_Multihead_256/all_zipcodes_with_tweets_bert_mean_predictions.xlsx
Binary predictions saved to Training_results/Beryl_5foldcv_BERT_Multihead_256/all_zipcodes_with_tweets_bert_binary_predictions.xlsx


In [ ]:
for model in models:
    del model

## Predict on Imelda

In [ ]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
import transformers
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import KFold
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score
import pandas as pd
import numpy as np
from tqdm import tqdm
import copy
import math

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.manual_seed(64)

# Load the saved models
save_path = 'Training_results/Imelda_5foldcv_BERT_Multihead_256/'
max_length = 256
batchsize = 48
saved_model_paths = [os.path.join(save_path, f"best_bert_model_fold_{i+1}.pth") for i in range(5)]
models = []

# Load BERT tokenizer and model
tokenizer = transformers.BertTokenizer.from_pretrained('bert-base-uncased')
model_bert = transformers.BertModel.from_pretrained('bert-base-uncased')

# Get the target list
target_list = [
    'homeOwnersInsurance', 'floodInsurance', 'destroyed', 'floodDamage', 'roofDamage',
    'tsaEligible', 'tsaCheckedIn', 'rentalAssistanceEligible', 'repairAssistanceEligible',
    'replacementAssistanceEligible', 'personalPropertyEligible'
]

def haversine(lat1, lon1, lat2, lon2):
    R = 3959
    lat1, lon1, lat2, lon2 = map(math.radians, [lat1, lon1, lat2, lon2])
    dlat, dlon = lat2 - lat1, lon2 - lon1
    a = math.sin(dlat / 2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon / 2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    return R * c

def get_box_area(lat1, lon1, lat2, lon2):
    side1 = haversine(lat1, lon1, lat1, lon2)
    side2 = haversine(lat1, lon1, lat2, lon1)
    return side1 * side2

# Get the target list if not provided
def get_target_list(target_list=[]):
    if not target_list:
        target_list = [
            'homeOwnersInsurance', 'floodInsurance', 'destroyed', 'floodDamage', 'roofDamage',
            'tsaEligible', 'tsaCheckedIn', 'rentalAssistanceEligible', 'repairAssistanceEligible',
            'replacementAssistanceEligible', 'personalPropertyEligible'
        ]
    return target_list

def grouped_tweets_to_dict(tweet_grouped):
    return {int(name): group['text'] for name, group in tweet_grouped}

# CustomDataset without labels (for inference only)
class InferenceDataset(Dataset):
    def __init__(self, texts, tokenizer, max_len=512):
        self.texts = texts
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_len,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )
        return {
            'input_ids': encoding['input_ids'].squeeze(0),
            'attention_mask': encoding['attention_mask'].squeeze(0)
        }

class BERTDeepMultiHeadClassifier(nn.Module):
    def __init__(self, num_targets=11, hidden_dim=256):
        super(BERTDeepMultiHeadClassifier, self).__init__()
        self.bert = model_bert
        self.drop = nn.Dropout(0.3)

        self.heads = nn.ModuleList([
            nn.Sequential(
                nn.Linear(self.bert.config.hidden_size, hidden_dim),
                nn.ReLU(),
                nn.Linear(hidden_dim, 1)
            ) for _ in range(num_targets)
        ])

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs.pooler_output
        x = self.drop(pooled_output)
        return torch.cat([torch.sigmoid(head(x)) for head in self.heads], dim=1)

# Load all trained models
for path in saved_model_paths:
    model = BERTDeepMultiHeadClassifier(num_targets=len(target_list)).to(device)
    model.load_state_dict(torch.load(path))
    model.eval()
    models.append(model)

size_threshold = 80

tweets2 = pd.read_csv('D:/TAMIDS/Disaster_Impact_Estimation/tweets/organized_with_zipcode.csv')  # read in massive tweets dataset
tweets_imelda = tweets2[(tweets2.storm_name == 'imelda')]
bboxes_useful = tweets_imelda.place_bbox.apply(lambda x: [[float(i.strip('(').strip(')')) for i in x.split(', ')][i] for i in [1,0,3,2]])
bbu_areas = bboxes_useful.apply(lambda x: get_box_area(*x))
tweets_imelda = tweets_imelda.loc[((tweets_imelda.geo.apply(lambda x: 'Point' in str(x))) | (bbu_areas < size_threshold)),:]  # since i'm using iloc i think the indices will match
tweet_grouped_imelda = tweets_imelda.groupby('zip_code')

tweet_dict = {int(name): '\n'.join(group['text'].to_list()) for name, group in tweet_grouped_imelda}

# Prepare data
zip_codes = list(tweet_dict.keys())
texts = [tweet_dict[zc] for zc in zip_codes]

# Create inference dataset and loader
inference_dataset = InferenceDataset(texts=texts, tokenizer=tokenizer, max_len=max_length)
inference_loader = DataLoader(inference_dataset, batch_size=batchsize, shuffle=False)

# Store predictions
all_preds = []

with torch.no_grad():
    for batch in tqdm(inference_loader, desc="Making predictions", unit="batch"):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)

        fold_preds = []
        for model in models:
            outputs = model(input_ids, attention_mask)
            fold_preds.append(outputs.cpu().numpy())

        fold_preds = np.stack(fold_preds, axis=0)  # Shape: (num_models, batch_size, num_targets)
        all_preds.append(fold_preds)

# Combine predictions
all_preds = np.concatenate(all_preds, axis=1)  # Shape: (num_models, total_samples, num_targets)
mean_preds = all_preds.mean(axis=0)            # Shape: (total_samples, num_targets)
binary_preds = (mean_preds > 0.5).astype(int)

target_list = [
    'homeOwnersInsurance', 'floodInsurance', 'destroyed', 'floodDamage', 'roofDamage',
    'tsaEligible', 'tsaCheckedIn', 'rentalAssistanceEligible', 'repairAssistanceEligible',
    'replacementAssistanceEligible', 'personalPropertyEligible'
]

# Save to files
mean_preds_df = pd.DataFrame(mean_preds, columns=target_list)
mean_preds_df.insert(0, "zip_code", zip_codes)
mean_preds_path = os.path.join(save_path, "all_zipcodes_with_tweets_bert_mean_predictions.xlsx")
mean_preds_df.to_excel(mean_preds_path, index=False)

binary_preds_df = pd.DataFrame(binary_preds, columns=target_list)
binary_preds_df.insert(0, "zip_code", zip_codes)
binary_preds_path = os.path.join(save_path, "all_zipcodes_with_tweets_bert_binary_predictions.xlsx")
binary_preds_df.to_excel(binary_preds_path, index=False)

print(f"Mean predictions saved to {mean_preds_path}")
print(f"Binary predictions saved to {binary_preds_path}")

C:\Users\udays\AppData\Local\Temp\ipykernel_18096\3976675152.py:110: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(path))
Making predictions

Mean predictions saved to Training_results/Imelda_5foldcv_BERT_Multihead_256/all_zipcodes_with_tweets_bert_mean_predictions.xlsx
Binary predictions saved to Training_results/Imelda_5foldcv_BERT_Multihead_256/all_zipcodes_with_tweets_bert_binary_predictions.xlsx
